In [16]:
import wandb
from math import isnan 
import pandas as pd
import numpy as np
import wandb
from tqdm import tqdm
from IPython.display import Markdown, display

    
api = wandb.Api()
project = "feas-ft"
workspace = "hounie"

# Get our two main experiments so far
experiment_tags = ["LIMA"]

# get all runs that both: 1.  match any experiment tag and 2. are finished
runs = api.runs(f"{workspace}/{project}",
                {"$and": [
                    {"tags": {"$in": experiment_tags}},
                    #{"state": "finished"}
                ]})


In [17]:
for run in runs:
    print(run.name)
    

ERM
Feas


In [48]:
io = IOArgs(
        checkpoint_dir="checkpoints/meta-llama/Llama-2-7b-hf",
        out_dir="out/lora/lima/dloads",
    )
io.checkpoint_dir

'checkpoints/meta-llama/Llama-2-7b-hf'

In [51]:
import os
import lightning as L
from lit_gpt.args import EvalArgs, IOArgs, TrainArgs
from lit_gpt.lora import GPT, Block, Config, lora_filter, mark_only_lora_as_trainable
from lit_gpt.tokenizer import Tokenizer
from lit_gpt.utils import (
    check_valid_checkpoint_dir,
    lazy_load,
)
from finetune.lora import get_longest_seq_length
from eval.lm_eval_harness import EvalHarnessBase
from pathlib import Path

def setup(run, checkpoints ="checkpoints/meta-llama/Llama-2-7b-hf",
           adapter_path = "downloads/", eval_tasks = ["arc_challenge", "piqa", "hellaswag"], num_fewshot=0):
    config = run.config
    io = IOArgs(
        checkpoint_dir=Path(checkpoints),
        out_dir=Path("out/lora/lima/dloads"),
    )
    '''
    train = TrainArgs(
        save_interval=config["save_interval"],
        log_interval=config["log_interval"],
        global_batch_size=config["global_batch_size"],
        micro_batch_size=config["micro_batch_size"],
        lr_warmup_steps=config["lr_warmup_steps"],
        epochs=config["epochs"],
        epoch_size=config["epoch_size"],
        learning_rate=config["learning_rate"],
        max_seq_length=config["max_seq_length"],
        algorithm=config["algorithm"],
        dual_learning_rate=config["dual_learning_rate"],
        resilient_learning_rate=config["resilient_learning_rate"],
        resilient_alpha=config["resilient_alpha"],
        tolerance=config["tolerance"],
    ),
    '''
    #eval = EvalArgs(interval=config["interval"], max_new_tokens=config["max_new_tokens"], max_iters=config["max_iters"])
    conf = Config.from_name(
            name=io.checkpoint_dir.name,
            r=config["r"],
            alpha=config["alpha"],
            dropout=config["dropout"],
            to_query=config["to_query"],
            to_key=config["to_key"],
            to_value=config["to_value"],
            to_projection=config["to_projection"],
            to_mlp=config["to_mlp"],
            to_head=config["to_head"],
        )
    fabric = L.Fabric(devices=1, strategy="auto", precision="bf16-true", plugins=None)
    fabric.seed_everything(0)  # same seed for every process to init model (FSDP)
    
    with fabric.init_module(empty_init=(False)):
        model = GPT(conf)
    #tokenizer = Tokenizer(io.checkpoint_dir)
    check_valid_checkpoint_dir(io.checkpoint_dir)

    if fabric.global_rank == 0:
        os.makedirs(io.out_dir, exist_ok=True)

    checkpoint_path = io.checkpoint_dir / "lit_model.pth"
    with lazy_load(io.checkpoint_path) as checkpoint:
        # strict=False because missing keys due to LoRA weights not contained in state dict
        model.load_state_dict(checkpoint, strict=False)
    with lazy_load(adapter_path) as checkpoint:
        model.load_state_dict(checkpoint, strict=False)
    if len(eval_tasks)> 0:
        eval_harness = EvalHarnessBase(
            checkpoint_dir=str(io.checkpoint_dir),
            model=model,
        )

    #results = eval_harness.run_eval(
    #    eval_tasks=["hendrycksTest-*"], num_fewshot=5, use_cache=False
    #)
    #wandb.log(results['results'])

    results = eval_harness.run_eval(
        eval_tasks=eval_tasks, num_fewshot=num_fewshot, use_cache=False
    )
    wandb.log(results['results'])




In [52]:
for run in runs:
    print(run)
    weights = run.file("out/lora/lima/lit_model_lora_finetuned.pth")
    # create the directory if it doesn't exist
    os.makedirs(f"download/{run.id}", exist_ok=True)
    out = weights.download(f"download/{run.id}", replace=True)
    print(out)
    setup(run, adapter_path=out)
    

<Run hounie/feas-ft/qi4gkpus (finished)>


Seed set to 0


<_io.TextIOWrapper name='download/qi4gkpus/out/lora/lima/lit_model_lora_finetuned.pth' mode='r' encoding='UTF-8'>


OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 47.53 GiB of which 27.94 MiB is free. Process 1730532 has 47.17 GiB memory in use. Including non-PyTorch memory, this process has 260.00 MiB memory in use. Of the allocated memory 0 bytes is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)